## **Sprint 3: H&M Personalized Fashion Recommendations**

### Part 2: BERT-Based Recommendations on Input

___
**Atoosa Rashid** 

[GitHub](https://github.com/atoosa-r/)

[LinkedIn](https://www.linkedin.com/in/atoosarashid/) 
___

### **Introduction**

In this data analysis, we explore H&M Group datasets, including transactions, customer information, and article details. H&M Group operates globally with 53 online markets and approximately 4850 stores. The objective is to uncover insights for developing effective product recommendations.

This notebook has been primarily created to facilitate an interactive UI experience, allowing participants to test and explore the recommendation system during our demo day presentations.

**Step-by-Step Plan**

1. Data Preparation

2. Word Embedding with BERT

3. Recommendation Function for Inputs

**Note:** This notebook was developed and optimized for Google Colab. For the best performance and compatibility, it is recommended to run it on Google Colab rather than Jupyter Notebooks or VS Code.
___

We'll begin by importing our necessary libraries and sentence transformers for word embedding.

In [1]:
#!pip install sentence_transformers

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.


In [1]:
# Importing libraries:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import re
import string

from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

import tensorflow as tf
import transformers
from transformers import DistilBertModel, DistilBertTokenizer

from sentence_transformers import SentenceTransformer


RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Failed to import transformers.data.data_collator because of the following error (look up to see its traceback):
module 'tensorflow' has no attribute 'data'

In [5]:
# Loading Transformer Model:

bert = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# Checking on bert:

bert

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [10]:
# Importing the df:

filtered_articles_df = pd.read_csv("filtered_articles.csv")

In [11]:
# Sanity check:

filtered_articles_df.head()

,article_id,prod_name,product_type_name,product_group_name,colour_group_name,department_name,index_group_name,section_name,garment_group_name,detail_desc,preprocessed_detail_desc,updated_description
0,108775015,Strap top,Vest top,Garment Upper body,Black,Jersey Basic,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,jersey top narrow shoulder straps,Jersey top with narrow shoulder straps. Color:...
1,108775044,Strap top,Vest top,Garment Upper body,White,Jersey Basic,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,jersey top narrow shoulder straps,Jersey top with narrow shoulder straps. Color:...
2,108775051,Strap top (1),Vest top,Garment Upper body,Off White,Jersey Basic,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,jersey top narrow shoulder straps,Jersey top with narrow shoulder straps. Color:...
3,110065001,OP T-shirt (Idro),Bra,Underwear,Black,Clean Lingerie,Ladieswear,Womens Lingerie,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",microfibre shirt bra underwired moulded lightl...,"Microfibre T-shirt bra with underwired, moulde..."
4,110065002,OP T-shirt (Idro),Bra,Underwear,White,Clean Lingerie,Ladieswear,Womens Lingerie,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",microfibre shirt bra underwired moulded lightl...,"Microfibre T-shirt bra with underwired, moulde..."


In [2]:
pd.set_option('display.max_rows', None)  # Set the option to display all rows
print(filtered_articles_df['product_type_name'].value_counts())
pd.reset_option('display.max_rows')  # Reset the option to default

NameError: name 'filtered_articles_df' is not defined

We don't need to include all these categories for visual purposes so we will exclude certain categories. Since we are showing a simple display of how our model works and some of these categories are not relevant. 

In [ ]:
# Defining a list of excluded categories:

excluded_categories = ['Washing bag', 'Dog wear', 'Long John', 'Sewing kit', 'Dog Wear', 'Umbrella','Gloves', 'Wireless earphone case', 'Marker pen','Costumes', 'Giftbox', 'Straw hat','Bra', 'Underwear bottom','Stain remover spray', 'Nipple covers','Wood balls','Waterbottle', 'Bikini top','Swimwear bottom', 'Underwear Tights', 'Underwear body', 'Underwear corset', 'Underwear set', 'Underdress', 'Bra extender']

# A new df excluding the specified categories:

filtered_articles_df_cleaned = filtered_articles_df[~filtered_articles_df['product_type_name'].isin(excluded_categories)]

# Viewing the entire list:

pd.set_option('display.max_rows', None)  # Set the option to display all rows
print(filtered_articles_df_cleaned['product_type_name'].value_counts())


In [ ]:
# Sanity Check:

filtered_articles_df_cleaned.head()

Our detailed description column does not include many important details regarding the articles because these details are seperated into their own individual columns. This includes:
- `prod_name`
- `product_type_name`
- `product_group_name`
- `colour_group_name`
- `department_name` 
- `index_group_name`
- `section_name`
- `garment_group_name` 

These details will help our reccomender system make more accurate reccomendations. In order to make our `detail_desc` column more detailed and accurate we will incorporate these specifics into the descriptions with the function shown below. 

In [ ]:
# Defining the function to create the updated description:

def create_updated_description(row):
    return (f"{row['prod_name']} - A {row['product_type_name']} ({row['product_group_name']}) in {row['colour_group_name']}. "
            f"For {row['department_name']} in {row['index_group_name']}, available in {row['section_name']}. "
            f"This product belongs to the {row['garment_group_name']} collection. Details: {row['detail_desc']}.")

# Applying the function and creating a new column with the updated descriptions:

filtered_articles_df_cleaned.loc[:, 'updated_description'] = filtered_articles_df_cleaned.apply(create_updated_description, axis=1)

In [ ]:
# Sanity Check:

filtered_articles_df_cleaned.head()

First, we will convert the article descriptions and their corresponding article IDs into lists. These lists will be used to get the embeddings and map the recommendations back to the original articles.

Next, we use our imported BERT model to generate embeddings for the article descriptions.

We then define a function, recommend_items, that takes an input sentence and returns the top most similar items based on their embeddings. The function works by encoding the input sentence, calculating cosine similarities with the article embeddings, identifying the top articles with the highest similarity scores, and creating a DataFrame containing the recommended article IDs and their similarity scores.

In [12]:
filtered_articles = filtered_articles_df['updated_description'].tolist()
article_ids = filtered_articles_df['article_id'].tolist()

#Getting embeddings using our bert model:

article_embeddings = bert.encode(filtered_articles)

def recommend_items(input_sentence, top_n=5):
    """
    Recommend top number of similar items based on an input sentence.

    Parameters:
    input_sentence (str): The input sentence for which recommendations are to be made.
    top_n (int, optional): The number of top similar items to recommend. Default is 5.

    Returns:
    pandas.DataFrame: A DataFrame containing the recommended article IDs and their similarity scores.
    """
    #Encoding the inputted sentence using the same bert model:
    input_embedding = bert.encode([input_sentence])[0]

    #Calculating cosine similarities between the input embedding and all article embeddings:
    similarities = cosine_similarity([input_embedding], article_embeddings)[0]

    #Getting the top similar items:
    top_indices = np.argsort(similarities)[::-1][:top_n]

    #Getting the top article IDs:
    top_article_ids = [article_ids[i] for i in top_indices]
    top_similarities = similarities[top_indices]

    #Creating a df for the recommendations:
    recommendations = pd.DataFrame({
        'article_id': top_article_ids,
        'similarity_score': top_similarities
    })

    return recommendations

We can now save the pre-trained BERT model, the generated article embeddings, and the article IDs to Google Drive. This ensures that we can reuse the model and embeddings in the future without needing to regenerate them, saving time and computational resources.

This step is necessary for deploying our model on Streamlit.

In [ ]:
import torch
from google.colab import drive
import pickle

# Mounting google drive:
drive.mount('/content/drive')

# Identifying the paths:
model_path = '/content/drive/My Drive/BERT_model'
embeddings_path = '/content/drive/My Drive/article_embeddings.npy'
ids_path = '/content/drive/My Drive/article_ids.npy'
articles_path = '/content/drive/My Drive/filtered_articles.pkl'

# Saving the bert model:
bert.save(model_path)

# Saving the article embeddings and IDs using numpy:
np.save(embeddings_path, article_embeddings)
np.save(ids_path, article_ids)

# Saving filtered_articles to a pickle file:
with open(articles_path, 'wb') as f:
    pickle.dump(filtered_articles, f)


With our function we can now input any sentence and receive the top reccomendations and the respective similarity_score.

In [13]:
#Sanity check 1:

input_sentence = "A casual summer dress"
top_recommendations = recommend_items(input_sentence, top_n=5)

print(top_recommendations)

   article_id  similarity_score
0   458239017          0.765280
1   502522006          0.728661
2   458239022          0.723263
3   458239001          0.723263
4   567484002          0.704251


In [16]:
#Sanity check 2:

input_sentence = "A simple black dress"
top_recommendations = recommend_items(input_sentence, top_n=5)

print(top_recommendations)

   article_id  similarity_score
0   629024002          0.764172
1   629024001          0.764172
2   577513003          0.752349
3   774506001          0.748125
4   879242010          0.744952


In [14]:
#Sanity check 3:

input_sentence = "A soft brown cardigan"
top_recommendations = recommend_items(input_sentence, top_n=5)

print(top_recommendations)

   article_id  similarity_score
0   807362001          0.782147
1   807362002          0.779681
2   807362003          0.779681
3   521846002          0.719355
4   937667002          0.708640


As shown in the example above, our generalized recommender system for inputted sentences is performing effectively. To enhance the user experience, the associated images of these articles will also be displayed in the Streamlit interface.